In [2]:
import datetime as dt
from dateutil.relativedelta import relativedelta

import pandas as pd
import numpy as np



In [3]:
WEEKDAY_NAMES_OPEN = ['Wednesday', 'Thursday', 'Sunday']
VOLUNTEERS_BY_WEEKDAY_NAME = {
    'Wednesday': ['Stephen', 'Sylvain', 'Andrew'],
    'Thursday': ['Finlay', 'Sanna'],
    'Sunday': ['Alex M', '__'],
    }

def build_roster(year, month, weekday_names_open=WEEKDAY_NAMES_OPEN, 
  volunteers_by_weekday_name=VOLUNTEERS_BY_WEEKDAY_NAME):
    """
    INPUTS:
    
    - ``year``: integer
    - ``month``: integer 1--12
    
    
    OUTPUTS:
    
    A pandas data frame
    """
    d1 = dt.date(year, month, 1)
    d2 = d1 + relativedelta(months=1)
    ix = pd.date_range(d1, d2, closed='left')
    f = pd.DataFrame(index=ix)
    f['date'] = f.index.strftime('%Y-%m-%d')
    f['weekday'] = f.index.weekday_name
    
    # Assign volunteers
    f['volunteers'] = f['weekday'].map(lambda x:
      ' + '.join(volunteers_by_weekday_name.get(x, ["__", "__"])))
    
    # Restrict to given weekdays
    cond = f['weekday'].isin(weekday_names_open)
    f = f[cond].copy()
    
    # Add notes. 
    # Staff meeting is on the first Wednesday of the month.
    f['notes'] = ''
    cond = f['weekday'] == 'Wednesday'
    cond &= f.index.day <= 7
    f.loc[cond, 'notes'] = 'Staff meeting 19:30--20:30'
    
    return f.reset_index(drop=True)

def get_year_month(roster, format="%Y%m"):
    """
    Given a roster, return the year and month it pertains to 
    in the given format.
    """
    return pd.to_datetime(roster['date']).dt.strftime(format).iat[0]

def to_markdown(df, path):
    """
    Save the given data frame as a Markdown table located
    at the given path.
    """
    # Get column names
    cols = df.columns

    # Create a new DataFrame with just the markdown
    # strings
    df2 = pd.DataFrame([['---',]*len(cols)], columns=cols)

    #Create a new concatenated DataFrame
    df3 = pd.concat([df2, df])

    #Save as markdown
    df3.to_csv(path, sep="|", index=False)


In [4]:
roster = build_roster(2016, 11)
print(roster)

date = get_year_month(roster)
path = 'roster_{!s}.md'.format(date)
to_markdown(roster, path)

          date    weekday                  volunteers  \
0   2016-11-02  Wednesday  Stephen + Sylvain + Andrew   
1   2016-11-03   Thursday              Finlay + Sanna   
2   2016-11-06     Sunday                 Alex M + __   
3   2016-11-09  Wednesday  Stephen + Sylvain + Andrew   
4   2016-11-10   Thursday              Finlay + Sanna   
5   2016-11-13     Sunday                 Alex M + __   
6   2016-11-16  Wednesday  Stephen + Sylvain + Andrew   
7   2016-11-17   Thursday              Finlay + Sanna   
8   2016-11-20     Sunday                 Alex M + __   
9   2016-11-23  Wednesday  Stephen + Sylvain + Andrew   
10  2016-11-24   Thursday              Finlay + Sanna   
11  2016-11-27     Sunday                 Alex M + __   
12  2016-11-30  Wednesday  Stephen + Sylvain + Andrew   

                         notes  
0   Staff meeting 19:30--20:30  
1                               
2                               
3                               
4                               
5  